<a id='Top'></a><br><br>
[Imports](#imports)<br> [Function List](#funclist)<br> [Apply band pass filter](#bpf)<br> [Create epochs using events](#epoching)<br> [Generate labels array from epochs](#labeling)<br>[Fit ICA to the epochs data](#ica)<br>[Apply ICA to the epochs data](#ica-apply)<br> [Apply standard scala](#sc)<br> [SVM](#svm)<br>[Model CNN](#createmodelCNN)<br>[Model CNN EEG-Conv-R](#createmodelCNN-R)<br>[Model RNN](#createmodelRNN)<br>[Model RNN convR](#createmodelRNNR)<br>[Confusion Matrix](#Confusionmarix)<br>[Define file paths](#filepaths)<br> [Load model saved in json file](#loadmodel)<br>[Fit the model](#fitmodel)<br>[Prediction](#prediction) 

<a id='imports'></a>
# Imports

In [ ]:
import mne
from mne.preprocessing import ICA
from mne.preprocessing import create_eog_epochs, create_ecg_epochs
from mne.decoding import Scaler

import numpy as np
from numpy import array
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Add,Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,Input,LSTM
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm 

import os.path 
from os import path
import pickle
import time

<a id='funclist'></a>
# Function list
<a href=#Top>Top</a>

In [ ]:
path_montage1 = r"path1path_montage2 = r"path2
path_list_montage = [path_montage1, path_montage2]

def loadfiles(path):
    import glob
    file_list = glob.glob(path)
    return file_list

    
def readcnt_noevent(cnt_file, event_id, duration):
    
    for montage_path in path_list_montage:
        if (path.exists(montage_path)):
            montage = mne.channels.read_montage('NuAmps40', path=montage_path)
    
    
    raw = mne.io.read_raw_cnt(cnt_file, montage, preload=True, verbose=0)

    # drop unwanted channels 


    raw = raw.drop_channels(['FP2', 'F3', 'F4', 'FT7', 'FCZ', 'FT8', 'C3', 'C4', 'TP7', 'CPZ', 'TP8',
                             'T5', 'PZ', 'T6', 'O1', 'O2', 'FT10', 'PO2'])

    raw = raw.drop_channels(['HEOR','VEOL'])  

    mne.io.set_bipolar_reference(raw, 'HEOL', 'VEOU', ch_name='EOG', copy=False)
    
    #raw.info['bads'] = ['F7', 'FZ', 'F8', 'FC4', 'A2']

     
    ###### add stimulus channel #########
    stim_data = np.zeros((1, len(raw.times)))
    info = mne.create_info(['STI'], raw.info['sfreq'], ['stim'])
    stim_raw = mne.io.RawArray(stim_data, info)
    raw.add_channels([stim_raw], force_update_info=True)

    raw.set_channel_types({'STI': 'stim'})
    
    ##### Setting fixed length events ########
    events = mne.make_fixed_length_events(raw, event_id , duration=duration)
    
    # Replacing the stim channel with events
    channel_index_no = len(raw.ch_names)-1

    for sample_index in events[:,0]:
        #print(sample_index)
        raw._data[channel_index_no, sample_index] = event_id
        
   
    return raw;

def raw_concat(raw1, raw2):
    # Concatenating raws
    from mne.io import  concatenate_raws
    if(raw1 == None):
        raw = raw2
    else:
        raws = [raw1, raw2]
        raw = concatenate_raws(raws)
    return raw;



# Save raw data into a single file
<a href=#Top>Top</a>

In [ ]:
# training sample path for path 1 
#path = training_data_path_home

# training sample path for path 2
#path = training_data_path_work 

def save_raw_data_single_file(folder_list):

    event_id = None
    raw = None

    for folder in folder_list:
        #print(folder)
        file_list = loadfiles(folder+"/*")
        for cnt_file in file_list:
           
            duration= 5
            # raw.info

            if(os.path.basename(cnt_file) == r"Fatigue state.cnt"):
                event_id = 1

            if(os.path.basename(cnt_file) == r"Normal state.cnt"):
                event_id = 2

            # Read the file 1
            
            raw1 = readcnt_noevent(cnt_file, event_id, duration)

            # Concatenating files
            raw = raw_concat(raw, raw1)

    outfile = open(output_file_path + r'\raw_data','wb')
    pickle.dump(raw,outfile)
    outfile.close()

# Read raw data file
<a href=#Top>Top</a>

In [ ]:
def read_raw_data_file(path):
    inputfile = open(path + r"\raw_data","rb")
    raw = pickle.load(inputfile)
    return raw


<a id='bpf'></a>
# Apply band pass filter
<a href=#Top>Top</a>

In [ ]:
def band_pass_filter(raw):
    raw.filter(4., 40., fir_design='firwin')
    return raw

<a id='epoching'></a>
# Create epochs using events
<a href=#Top>Top</a>

In [ ]:
def epoching(raw, event_ids):
    
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude=[])
    
    events = mne.find_events(raw, stim_channel='STI', shortest_event=1)
    
    print('raw: ', raw.ch_names)
    
    # define an epoch size
    tmin, tmax = -0.0, 0.499

    # reject epochs which have their values over 150 micro volts
    reject = None#dict(eeg=150e-6)

    # create epochs
    epochs = mne.Epochs(raw, events, event_id=event_ids, baseline=(0,0), tmin=tmin, tmax=tmax, 
                        preload=True, reject=reject, picks=picks)
    #evoked = epochs.average()
    print('epochs: ', epochs.ch_names)
    epochs.plot(title='original')
    

    
    return epochs

<a id='labeling'></a>
# Generate labels array from epochs 
<a href=#Top>Top</a>

In [ ]:
def labeling(epochs):
    # this list stores label of each epoch
    epochs_labels = []

    i = 0
    epochs_length = epochs.__len__()
    while i < epochs_length:
        full_label_id = epochs[i].event_id
        #print(full_label_id)
        if ('1' in full_label_id.keys()):
            epochs_labels.append(1)
        elif ('fatigue_state' in full_label_id.keys()) :
            epochs_labels.append(1)
        else:
            epochs_labels.append(0)
        i = i+1
     
    # label format suitable for the 'loss' function
    from tensorflow.keras.utils import to_categorical
    categoric_epochs_labels = to_categorical(epochs_labels, num_classes=2)
    
    print(categoric_epochs_labels[1], epochs_labels[1])
        
    return categoric_epochs_labels, epochs_labels

<a id='ica'></a>
# Fit ICA to the epochs data
<a href=#Top>Top</a>

In [ ]:
def ica_preprocessing(epochs):
    %matplotlib
    
    picks_eeg = mne.pick_types(epochs.info, meg=False, eeg=True, eog=False,
                           stim=False, exclude='bads')
    
    #n_components = 25  # if float, select n_components by explained variance of PCA
    method = 'fastica'  
    decim = 3  

    random_state = 23
    
    reject = dict(eeg=200e-6)
    
    ica = ICA(method=method, random_state=random_state)
    
    ica.fit(epochs.drop_bad(reject=reject), decim=decim)
    

    
    return epochs,ica

<a id='ica-apply'></a>
# Apply ICA
<a href=#Top>Top</a>

In [ ]:
def apply_ica(ica_epochs):
    eog_average = ica_epochs.average()
    eog_inds, scores = ica.find_bads_eog(ica_epochs, ch_name='EOG' ) 

    ica.plot_scores(scores, exclude=eog_inds) 


    ica.plot_sources(eog_average, exclude=eog_inds) 

    if len(eog_inds) !=0 :
        ica.plot_properties(ica_epochs, picks=eog_inds, psd_args={'fmax': 35.},
                    image_args={'sigma': 1.})

   

    ica.plot_overlay(eog_average, exclude=eog_inds, show=False)

    ica.exclude.extend(eog_inds)


<a id='sc'></a>
# Apply standard scalar
<a href=#Top>Top</a>

In [ ]:
def z_score(epochs, labels=None):
    epochs_data = epochs.get_data()
    
    print(epochs_data.shape)
    
    h = epochs_data.shape[1]-1
    w = epochs_data.shape[2]

    # Remove EOG channel and reshape to fit the tensorflow input
    epochs_data = epochs_data [:, 1:,:]
    
    print(epochs_data.shape)
    
    scaler = Scaler(info=None, scalings='mean')
    epochs_scaled = scaler.fit_transform(epochs_data,labels)
    print(epochs_scaled.shape)
    
    epochs_scaled = epochs_scaled.reshape(-1,h,w,1)
    
 
    return epochs_scaled

<a id='svm'></a>
# SVM
<a href=#Top>Top</a>

In [ ]:
# accept 2D array and labels

def SVM_Classify(feature_vectors_final, labels):
    #print(feature_vectors_final.shape)

    X_train, X_test, y_train, y_test = train_test_split(feature_vectors_final, labels, test_size=0.4, random_state=0);


    clf = svm.SVC(kernel='rbf', C=2).fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    print("Accuracy is %f" %clf.score(X_test, y_test));
    return clf

<a id='createmodelCNN'></a>
# CNN model layout
<a href=#Top>Top</a>

In [ ]:
def create_CNN_model(input_shape):

    # Always have to give a 4D array as input to the CNN
    visible = Input(input_shape[1:])

    conv1 = Conv2D(32, (5,5), activation='relu', strides=1, bias_initializer='zeros')(visible)
    # model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

    lrn2 = tf.nn.local_response_normalization(conv1)

    conv3 = Conv2D(64, (3,3), strides=1, activation='relu', bias_initializer='zeros')(lrn2)
    # model.add(MaxPooling2D(pool_size=(2,2), padding='same',))

    conv4 = Conv2D(32, (3,3), activation='relu', strides=1, bias_initializer='zeros')(conv3)

    pool5 = MaxPooling2D(pool_size=(2,2), strides = 2)(conv4)

    #flat1 = Flatten()(pool5)
    dense1 = Dense(2048)(pool5)
    fc6 = Dropout(0.5)(dense1)
    flat1 = Flatten()(fc6)

    output = Dense(2, activation='softmax')(flat1)

    model = Model(inputs=visible, outputs=output)
    print(model.summary())
    plot_model(model, to_file=r'model_plot.png', show_shapes=True, show_layer_names=True)

    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    return model


<a id='createmodelCNN-R'></a>
# CNN EEG-Conv-R model
<a href=#Top>Top</a>

In [ ]:
def identity_block(X):

    # Save the input value. need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    
    
    X = Conv2D(32, (5,5), strides=1, activation='relu', bias_initializer='zeros', padding = 'same')(X)
    activation1 = Activation('relu')(X)
    

    
    # Second component of main path (≈3 lines)
    
    
    X = Conv2D(32, (3,3), strides=1, bias_initializer='zeros', padding = 'same')(X)

    
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:
def create_CNN_model_R(input_shape):

    # Always have to give a 4D array as input to the CNN
    visible = Input(input_shape[1:])

    X = Conv2D(32, (3,3), activation='relu', strides=3, bias_initializer='zeros', padding = 'same')(visible)
    #print(X.shape)
    
    X = identity_block(X)
    
    lrn = tf.nn.local_response_normalization(X)

    X = identity_block(lrn)

    #pool5 = MaxPooling2D(pool_size=(2,2), strides = 2)(conv4)

    dense = Dense(4096)(X)
    fc = Dropout(0.5)(dense)
    flat = Flatten()(fc)

    output = Dense(2, activation='softmax')(flat)

    model = Model(inputs=visible, outputs=output)
    #print(model.summary())
    plot_model(model, to_file=r'model_plot_R.png', show_shapes=True, show_layer_names=True)

    print(model.summary())
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    return model

<a id='createmodelRNN'></a>
# RNN model layout
<a href=#Top>Top</a>

In [ ]:
def create_RNN_model(input_shape):
    
    #model = Sequential()
    
    # The input to every LSTM layer must be three-dimensional.
    visible = Input(input_shape[1:-1])
    
    lstm1 = LSTM(512, return_sequences=True)(visible)
    
    lstm2 = LSTM(1024, return_sequences=True)(lstm1) 
    
    lstm3 = LSTM(2048, return_sequences=True)(lstm2)
    
    flat1 = Flatten()(lstm3)
    
    dense1 = Dense(512)(flat1)
    
    output = Dense(2, activation='softmax')(dense1)

    model = Model(inputs=visible, outputs=output)
    print(model.summary())
    plot_model(model, to_file=r'model_plot.png', show_shapes=True, show_layer_names=True)

    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    return model

<a id='createmodelRNNR'></a>
# RNN ConvR model layout
<a href=#Top>Top</a>

In [ ]:
def identity_block_rnn(rnn_width,X):

    # Save the input value. need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = LSTM(rnn_width, return_sequences=True)(X)
    # padded_X = tf.keras.preprocessing.sequence.pad_sequences(X,padding='post')
    
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    
    
    return X

In [ ]:
def create_RNN_model_R(input_shape):
    
    # The input to every LSTM layer must be three-dimensional.
    visible = Input(input_shape[1:-1])
    
    lstm1 = LSTM(512, return_sequences=True)(visible)
    
    X = identity_block_rnn(1024, lstm1) 
    
    X = identity_block_rnn(2048, X)
    
    flat1 = Flatten()(X)
    
    dense1 = Dense(512)(flat1)
    
    output = Dense(2, activation='softmax')(dense1)

    model = Model(inputs=visible, outputs=output)
    print(model.summary())
    plot_model(model, to_file=r'model_plot.png', show_shapes=True, show_layer_names=True)

    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    return model

<a id='Confusionmarix'></a>
### Confusion marix
<a href=#Top>Top</a>

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):

    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     fontsize=16,
                     weight='bold',
                     color="brown" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     fontsize=16,
                     weight='bold',
                     color="brown" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

<a id='filepaths'></a>
# Define file paths
<a href=#Top>Top</a>

In [ ]:
data_path1 = r"D:\EEG\5202739"
data_path2 = r"path_1
data_path_list = [data_path1, data_path2]

for data_path in data_path_list:
        if (path.exists(data_path)):
            training_folder_list = loadfiles(data_path + r"\Training\*")
            testing_folder_list = loadfiles(data_path + r"\Testing\*")
            output_file_path = data_path


In [ ]:
#save_raw_data_single_file(training_folder_list)
training_raw_data = read_raw_data_file(output_file_path)
raw = band_pass_filter(training_raw_data)
event_ids = dict(fatigue_state=1, normal_state=2)
training_epochs = epoching(raw, event_ids)

ica_epochs, ica = ica_preprocessing(training_epochs)
apply_ica(ica_epochs)

epochs_copy = ica_epochs.copy()
ica.apply(ica_epochs)


training_labels_categorical, training_labels = labeling(training_epochs)
# print(training_labels)
training_epochs.get_data().shape

standardized_epochs = z_score(ica_epochs, training_labels)
input_shape = standardized_epochs.shape
print("standardize epochs shape : ",standardized_epochs.shape)
print("input shape : ",input_shape[:-1])

In [ ]:
from math import modf

std_shape = standardized_epochs.shape

print(standardized_epochs.reshape(std_shape[0], std_shape[1], std_shape[2]).shape)
list1 = np.asarray(standardized_epochs[:,:,:,0]) 
training_array = []
validation_array = []
i = 0
j = modf(std_shape[0]*2/3)[1]
if i < j:
    while i < j:
        training_array.append(list1[i,:,:])
        i = i+1
if i >= j:
    while i < std_shape[0]:
        validation_array.append(list1[i,:,:])
        i = i+1
        
training_array = np.asarray(training_array)
validation_array = np.asarray(validation_array)
print(training_array.shape)
print(validation_array.shape)


    

In [ ]:
keys = ['train', 'validation']
values = [training_array, validation_array]
predintion_dictionary = dict(zip(keys, values))
print(predintion_dictionary['validation'].shape)

<a id=#fitmodel></a>
# Fit the model
<a href=#Top>Top</a>

[Call backs](#cb)<br>[cnn fit](#cnnfit)<br>[cnn convR fit](#cnnRfit)<br>[rnn fit](#rnnfit)<br>[rnn convR fit](#rnnRfit)<br>

### SVM model fit

In [ ]:
# reshape the standardized data into 2d

# windows, channels, samples = standardized_epochs[:,:,:,0].shape
# feature_vectors_final = standardized_epochs.reshape((windows, channels*samples))

feature_vectors_final = standardized_epochs.reshape(len(standardized_epochs[:,:,:,0]),-1)
print("feature vector final", feature_vectors_final.shape)
print(np.asarray(training_labels).shape)
model = SVM_Classify(feature_vectors_final, training_labels)

In [ ]:
from sklearn.model_selection import validation_curve
param_range = np.logspace(-6, -1, 5)
train_scores, test_scores = validation_curve(
    svm.SVC(kernel='rbf', C=2), feature_vectors_final, training_labels, param_name="gamma", param_range=param_range,
    scoring="accuracy", n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with SVM")
plt.xlabel(r"$\gamma$")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

<a id="cb"></a>
#### Callbacks
<a href=#Top>Top</a>

In [ ]:
# modelcheckpoint
checkpoint_prefix_cnn = "..\model_CNN_{epoch}"
checkpoint_prefix_cnn_convR = "..\model_CNN_convR_{epoch}"

checkpoint_prefix_rnn = "..\model_RNN_{epoch}"
checkpoint_prefix_rnn_convR = "..\model_RNN_convR_{epoch}"

# SAVE the weights for the model in the epoch with the lowest val_loss
checkpoint_callback_cnn = ModelCheckpoint(filepath=checkpoint_prefix_cnn, save_best_only=True, 
                                     save_weights_only=True, monitor='val_accuracy', verbose=2)

checkpoint_callback_cnn_convR = ModelCheckpoint(filepath=checkpoint_prefix_cnn_convR, save_best_only=True, 
                                     save_weights_only=True, monitor='val_accuracy', verbose=2)

checkpoint_callback_rnn = ModelCheckpoint(filepath=checkpoint_prefix_rnn, save_best_only=True, 
                                     save_weights_only=True, monitor='val_accuracy', verbose=2)

checkpoint_callback_rnn_convR = ModelCheckpoint(filepath=checkpoint_prefix_rnn_convR, save_best_only=True, 
                                     save_weights_only=True, monitor='val_accuracy', verbose=2)

# tensorboard
local_time = time.localtime(time.time())
logs_folder = "%s-%s" % (local_time.tm_mon,local_time.tm_mday)
time_format = "%s.%s" % (local_time.tm_hour,local_time.tm_min)

NAME_CNN = r"..\logs\%s\logs_cnn_%s" % (logs_folder,time_format)
tensorboard_cnn = TensorBoard(log_dir=NAME_CNN)

NAME_CNN_convR = r"..\logs\%s\logs_cnn_convR_%s" % (logs_folder,time_format)
tensorboard_cnn_convR = TensorBoard(log_dir=NAME_CNN_convR)

NAME_RNN =  r"..\logs\%s\logs_rnn_%s" % (logs_folder,time_format)
tensorboard_rnn = TensorBoard(log_dir=NAME_RNN)

NAME_RNN_convR =  r"..\logs\%s\logs_rnn_convR_%s" % (logs_folder,time_format)
tensorboard_rnn_convR = TensorBoard(log_dir=NAME_RNN_convR)

# callbacks array
callbacks_cnn = [checkpoint_callback_cnn, tensorboard_cnn]

callbacks_cnn_convR = [checkpoint_callback_cnn_convR, tensorboard_cnn_convR]

callbacks_rnn = [checkpoint_callback_rnn, tensorboard_rnn]

callbacks_rnn_convR = [checkpoint_callback_rnn_convR, tensorboard_rnn_convR]

In [ ]:
# seperate traning and testing data

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(standardized_epochs, training_labels_categorical, test_size=0.33)

<a id="cnnfit"></a>
### CNN model fit
<a href=#Top>Top</a><br>
<a href=#cb>Call backs</a>

In [ ]:
history_cnn = create_CNN_model(input_shape).fit(X_train,Y_train, batch_size=10, epochs=30, validation_split=0.1,
                    callbacks=callbacks_cnn)



In [ ]:
model = create_CNN_model(input_shape)
latest_model = tf.train.latest_checkpoint(r'..\CNN_checkpoint')
print(latest_model)
model.load_weights(latest_model)

score = model.evaluate(X_test, Y_test, verbose=1)
print('test loss, test acc:', score)

<a id="cnnRfit"></a>
### CNN_convR model fit
<a href=#Top>Top</a><br>
<a href=#cb>Call backs</a>

In [ ]:
print(input_shape)

history_cnn_convR =  create_CNN_model_R(input_shape).fit(X_train,Y_train, batch_size=10, epochs=30, validation_split=0.1,
                    callbacks=callbacks_cnn_convR)

In [ ]:
model = create_CNN_model_R(input_shape)
latest_model = tf.train.latest_checkpoint(r'..\CNN_convR_checkpoInt')
print(latest_model)
model.load_weights(latest_model)

score = model.evaluate(X_test, Y_test, verbose=1)
print('test loss, test acc:', score)

<a id=rnnfit></a>
### RNN model fit
<a href=#Top>Top</a><br>
<a href=#cb>Call backs</a>

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(np.squeeze(standardized_epochs, axis=3), training_labels_categorical, test_size=0.33)

In [ ]:
history_rnn = create_RNN_model(input_shape).fit(X_train,Y_train, batch_size=10, epochs=30, validation_split=0.1,
                    callbacks=callbacks_rnn)

In [ ]:
model = create_RNN_model(input_shape)
latest_model = tf.train.latest_checkpoint(r'..\RNN_checkpoInt')
print(latest_model)
model.load_weights(latest_model)

score = model.evaluate(X_test, Y_test, verbose=1)
print('test loss, test acc:', score)

<a id=rnnRfit></a>
### RNN convR model fit
<a href=#Top>Top</a><br>
<a href=#cb>Call backs</a>

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(np.squeeze(standardized_epochs, axis=3), training_labels_categorical, test_size=0.33)

history_rnn = create_RNN_model_R(input_shape).fit(X_train,Y_train, batch_size=20, epochs=20, validation_split=0.1,
                    callbacks=callbacks_rnn)

## Evaluate model

In [ ]:
print(tf.train.latest_checkpoint('..\modelcheckpoint'))

# Save model to jason
<a href=#Top>Top</a>

In [ ]:
def save_model(model,model_path, weight_path):
    # serialize model to JSON
    model_CNN = model.to_json()
    with open(model_path, "w") as json_file:
        json_file.write(model_CNN)
    # serialize weights to HDF5
    model.save_weights(weight_path)
    print("Saved model to disk")

In [ ]:
model_path = r"..\model_CNN.json"
weight_path = r"..\model_weights.h5"
save_model(model, model_path, weight_path)

<a id='loadmodel'></a>
# Load model saved in json file
<a href=#Top>Top</a>

In [ ]:
def load_saved_model(model_path, weight_path):
    # load json and create model
    from tensorflow.keras.models import model_from_json
    json_file = open(model_pathr, 'r')
    loaded_model_CNN = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_CNN)
    # load weights into new model
    loaded_model.load_weights(weight_pathr)
    print("Loaded model from disk")
    
    return loaded_model

<a id='prediction'> </a>
# Predictions 
<a href=#Top>Top</a>

[cnn predict](#cnnpredict)<br>[cnn_convR predict](#cnnRpredict)<br>[rnn predict](#rnnpredict)<br>[svm predict](#svmpredict)<br>

Run the cells below before <br>
[Imports](#imports)<br> [Function List](#funclist)<br> [Apply band pass filter](#bpf)<br> [Create epochs using events](#epoching)<br> [Generate labels array from epochs](#labeling)<br>[Fit ICA to the epochs data](#ica)<br>[Apply ICA to the epochs data](#ica-apply)<br> [Apply standard scala](#sc)<br> [Define file paths](#filepaths)<br> [Load model saved in json file](#loadmodel) 

In [ ]:
event_id = None
raw = None
data_file_list = []
testing_label_list = []
pred_class = []

for folder in testing_folder_list:
   
    file_list = loadfiles(folder+"/*")
    for cnt_file in file_list:
        duration= 5

        print(os.path.abspath(cnt_file))
        
        if(os.path.basename(cnt_file) == r"Fatigue state.cnt"):
            event_id = 1

        if(os.path.basename(cnt_file) == r"Normal state.cnt"):
            event_id = 2
        
        # Read the file
        test_raw = readcnt_noevent(cnt_file, event_id, duration)
        
        bp_raw = band_pass_filter(test_raw)
        
        testing_epochs = epoching(bp_raw, event_id)
        
        #print(event_id,testing_labels)
        
        ica_testing_epochs, ica = ica_preprocessing(testing_epochs)
        
        apply_ica(ica_testing_epochs)
        
        testing_labels_binary, testing_labels = labeling(ica_testing_epochs)
        
        #print(testing_labels)
        
        standardized_testing_epochs = z_score(ica_testing_epochs)
        
        if(len(data_file_list) == 0 and len(testing_label_list) == 0):
            data_file_list = standardized_testing_epochs
            testing_label_list = testing_labels
        else:
            data_file_list = np.append(data_file_list, standardized_testing_epochs ,axis=0)
            testing_label_list = np.append(testing_labels,testing_label_list)
            
        print("testing data shape " , data_file_list.shape)
        print(np.asarray(testing_label_list).shape)

# Create a new model instance
input_shape = standardized_testing_epochs.shape


#  score = model.evaluate(standardized_testing_epochs, testing_labels_binary, verbose=1)
#  print("Accuracy: ", score[1])

#         # calculate the accuracy 
#         k = 0
#         i = 0
#         if(prediction.shape[0] != np.asarray(testing_labels).shape[0]):
#             print("predicted set size is diffrent from testing labels size")
#         else:
#             while(i<prediction.shape[0]):
#                 if(prediction[i]==testing_labels[i]):
#                     k = k+1
#                 i = i+1
            
                
#         print(k, np.asarray(testing_labels).shape[0])                  
#         accuracy = k/np.asarray(testing_labels).shape[0]
                          
#         print("Prediction accuracy = ",accuracy)        

<a id="cnnpredict"></a>
### CNN predict
<a href=#Top>Top</a>

In [ ]:
#-------------------  CNN  ---------------------#    

model = create_CNN_model(input_shape)
# load the latest model saved by model checkpoint with lowest val_loss
latest_model = tf.train.latest_checkpoint(r'..\CNN_checkpoInt')

predict_input = data_file_list

<a id="cnnRpredict"></a>
### CNN convR predict
<a href=#Top>Top</a>

In [ ]:
#-------------------  CNN-convR  ---------------------#    

model = create_CNN_model_R(input_shape)
# load the latest model saved by model checkpoint with lowest val_loss
latest_model = tf.train.latest_checkpoint(r'..\CNN_convR_checkpoInt')

predict_input = data_file_list 

<a id="rnnpredict"></a>
### RNN predict
<a href=#Top>Top</a>

In [ ]:
#-------------------  RNN  ---------------------#

model = create_RNN_model(input_shape)
latest_model = tf.train.latest_checkpoint(r'..\RNN_checkpoInt')

predict_input = data_file_list[:,:,:,0]

### Predicted class list from CNN and RNN

In [ ]:
print(latest_model)
model.load_weights(latest_model)

# get the predicted class labesl
if(len(pred_class) == 0):
    pred_class = model.predict(predict_input).argmax(axis=-1)
else:
    pred_class = np.append(pred_class,model.predict(predict_input).argmax(axis=-1))

<a id="svmpredict"></a>
### SVM predict
<a href=#Top>Top</a>

In [ ]:
predict_data = data_file_list.reshape(len(data_file_list[:,:,:,0]),-1)

pred_class = model.predict(predict_data)

In [ ]:
print(np.array(testing_label_list).shape, np.array(pred_class).shape)
cm=confusion_matrix(testing_label_list,pred_class,labels=[0, 1])
print(cm)
plot_confusion_matrix(cm=cm, 
              normalize    = False,
              target_names = ['Normal','Fatigue'],
              title        = "Confusion Matrix")

In [ ]:
len(testing_label_list)

In [ ]:
len(pred_class)

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=1)
print("Accuracy: ", score[1])

# Plot Accuracy / Loss
<a href=#Top>Top</a>

In [ ]:
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.1, 1.5])
# plt.legend(loc='lower right')

# plt.plot(history_rnn.history['loss'], label='loss')
# plt.plot(history_rnn.history['val_loss'], label = 'val_loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.ylim([0.0, 3.0])
# plt.legend(loc='lower right')

plt.plot(model.history['loss'], label='loss')
plt.plot(model.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0.0, 3.0])
plt.legend(loc='lower right')